<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao29_seq2seq_encdec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento de Transformer para Tradução PT -> EN

Notebook completo que implementa um modelo Transformer "do zero" (usando as camadas de atenção do Keras) para treinar uma tradução de Português para Inglês, utilizando o conjunto de dados do Anki.

Etapas:
1.  **Configuração e Imports**
2.  **Download e Preparação dos Dados**
3.  **Vetorização e Tokenização**
4.  **Criação do Pipeline `tf.data`**
5.  **Construção do Modelo Transformer**
6.  **Treinamento**
7.  **Função de Inferência (Tradução)**

In [12]:
# Célula 1: Instalações e Imports
# (Nenhuma instalação extra é necessária no Colab para isso)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import string
import re
import random

In [28]:
## -----------------------------------------------------------------
## Célula 2: Configuração dos Hiperparâmetros
## -----------------------------------------------------------------
# Para um demo rápido no Colab, usamos um subconjunto dos dados
# e hiperparâmetros menores.
# O dataset completo tem ~190.000 pares.
NUM_SAMPLES = 100000
# Hiperparâmetros do Modelo
EMBED_DIM = 256        # Dimensão dos embeddings
NUM_HEADS = 4          # Número de cabeças de atenção
FF_DIM = 512           # Dimensão da camada Feed-Forward interna
NUM_ENCODER_LAYERS = 2 # Número de camadas do Encoder
NUM_DECODER_LAYERS = 2 # Número de camadas do Decoder
VOCAB_SIZE = 15000     # Tamanho do vocabulário
MAX_SEQ_LENGTH = 30    # Comprimento máximo das sentenças

# Hiperparâmetros de Treinamento
BATCH_SIZE = 64
EPOCHS = 30  # Para um treinamento real, aumente para 30-50

In [30]:
## -----------------------------------------------------------------
## Célula 3: Download e Preparação dos Dados
## -----------------------------------------------------------------

print("Baixando e extraindo o dataset do Anki (por-eng)...")
!wget -q http://www.manythings.org/anki/por-eng.zip
!unzip -q por-eng.zip

data_path = "por.txt"
print("Dataset baixado.")

# Leitura do arquivo
# O formato é: Inglês \t Português \t Atribuição
# Queremos: Português (entrada) -> Inglês (saída)
text_pairs = []
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[:-1]:  # O último é vazio
    if "\t" not in line:
        continue
    parts = line.split("\t")
    if len(parts) < 2:
        continue

    # Nosso objetivo é Português -> Inglês
    target_text = parts[0] # Inglês
    input_text = parts[1]  # Português

    text_pairs.append((input_text, target_text))

print(f"Total de pares de sentenças: {len(text_pairs)}")

# Embaralhar e selecionar um subconjunto
random.shuffle(text_pairs)
print(f"Usando {NUM_SAMPLES} amostras para este demo.")
train_pairs = text_pairs[: int(NUM_SAMPLES * 0.9)]
val_pairs = text_pairs[int(NUM_SAMPLES * 0.9) : NUM_SAMPLES]

print(f"\nExemplo de par (Input, Target):")
print(f"Input (PT): {train_pairs[0][0]}")
print(f"Target (EN): {train_pairs[0][1]}")

Baixando e extraindo o dataset do Anki (por-eng)...
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Dataset baixado.
Total de pares de sentenças: 196620
Usando 100000 amostras para este demo.

Exemplo de par (Input, Target):
Input (PT): Tom disse que ele não entendeu nada.
Target (EN): Tom said he didn't understand anything.


In [31]:
## -----------------------------------------------------------------
## Célula 4: Pré-processamento e Vetorização do Texto
## -----------------------------------------------------------------

# Precisamos de uma função de padronização customizada para:
# 1. Colocar em minúsculas
# 2. Remover pontuação
# 3. Adicionar tokens [START] e [END] (APENAS para o alvo/target)

strip_chars = string.punctuation.replace("[", "").replace("]", "")

def custom_standardization(input_string):
    # Minúsculas e remoção de HTML (embora não deva ter aqui)
    lowercase = tf.strings.lower(input_string)

    # Remove pontuação
    no_punctuation = tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    )

    # Adiciona tokens de início e fim
    # Isso é crucial para o decoder
    return tf.strings.join(["[START]", no_punctuation, "[END]"], separator=" ")

# Camada de Vetorização para a ENTRADA (Português)
source_vectorization = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_SEQ_LENGTH,
    # Não adicionamos [START]/[END] na entrada
)

# Camada de Vetorização para o ALVO (Inglês)
target_vectorization = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_SEQ_LENGTH + 1, # +1 para acomodar [START] ou [END]
    standardize=custom_standardization,
)

# "Adaptar" (treinar) os vocabulários
print("\nAdaptando vocabulários...")
source_texts = [pair[0] for pair in train_pairs]
target_texts = [pair[1] for pair in train_pairs]

source_vectorization.adapt(source_texts)
target_vectorization.adapt(target_texts)
print("Vocabulários adaptados.")

print(f"\nVocabulário PT (Fonte): {source_vectorization.get_vocabulary()[:10]}...")
print(f"Vocabulário EN (Alvo): {target_vectorization.get_vocabulary()[:10]}...")


Adaptando vocabulários...
Vocabulários adaptados.

Vocabulário PT (Fonte): ['', '[UNK]', np.str_('tom'), np.str_('que'), np.str_('o'), np.str_('não'), np.str_('eu'), np.str_('de'), np.str_('a'), np.str_('você')]...
Vocabulário EN (Alvo): ['', '[UNK]', np.str_('[START]'), np.str_('[END]'), np.str_('tom'), np.str_('i'), np.str_('to'), np.str_('you'), np.str_('the'), np.str_('a')]...


In [32]:
## -----------------------------------------------------------------
## Célula 5: Criação do Pipeline `tf.data`
## -----------------------------------------------------------------

# Esta função prepara os dados para o formato que o Transformer espera:
# X = (encoder_inputs, decoder_inputs)
# Y = decoder_outputs (que é o decoder_inputs deslocado em 1)

def format_dataset(source, target):
    source_vec = source_vectorization(source)
    target_vec = target_vectorization(target)

    # target_vec é "[START] ... [END]"
    # decoder_inputs é "[START] ..." (sem o [END])
    decoder_inputs = target_vec[:, :-1]

    # decoder_outputs é "... [END]" (sem o [START])
    decoder_outputs = target_vec[:, 1:]

    # O modelo receberá dois inputs e tentará prever o output
    return (
        {"encoder_inputs": source_vec, "decoder_inputs": decoder_inputs},
        decoder_outputs
    )

def make_dataset(pairs):
    source_texts, target_texts = zip(*pairs)
    source_texts = list(source_texts)
    target_texts = list(target_texts)

    dataset = tf.data.Dataset.from_tensor_slices((source_texts, target_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(tf.data.AUTOTUNE).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

print("\nFormato do Lote de Dados (X, Y):")
for batch in train_ds.take(1):
    X, Y = batch
    print("X['encoder_inputs'] shape:", X['encoder_inputs'].shape)
    print("X['decoder_inputs'] shape:", X['decoder_inputs'].shape)
    print("Y (decoder_outputs) shape:", Y.shape)


Formato do Lote de Dados (X, Y):
X['encoder_inputs'] shape: (64, 30)
X['decoder_inputs'] shape: (64, 30)
Y (decoder_outputs) shape: (64, 30)


In [33]:
## -----------------------------------------------------------------
## Célula 6: Construção dos Blocos do Transformer
## -----------------------------------------------------------------

# O Transformer é composto por um Encoder e um Decoder.
# Ambos usam Atenção Multi-Cabeça (Multi-Head Attention).

### 6.1 - Embedding de Posição
# O Transformer não tem recorrência (como RNNs), então precisamos
# injetar informação sobre a posição dos tokens.
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        # Embedding dos tokens
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        # Embedding da posição (um vetor para cada posição)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen
        self.embed_dim = embed_dim

    def call(self, x):
        # Cria um vetor de posições (0, 1, 2, ..., maxlen-1)
        # tf.shape(x)[1] é o comprimento da sequência
        positions = tf.range(start=0, limit=tf.shape(x)[1], delta=1)

        # Obtém os embeddings de posição
        position_embeddings = self.pos_emb(positions)

        # Obtém os embeddings de token
        token_embeddings = self.token_emb(x)

        # Soma os dois
        return token_embeddings + position_embeddings

    def get_config(self):
        config = super().get_config()
        config.update({
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

### 6.2 - Camada do Encoder Transformer
class TransformerEncoderLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim

        # Camada de Atenção Multi-Cabeça (Self-Attention)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

        # Rede Feed-Forward
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

        # Camadas de Normalização (Layer Normalization)
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        # Camadas de Dropout
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training=False):
        # 1. Self-Attention + Conexão Residual
        # (query=inputs, key=inputs, value=inputs)
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        # Conexão residual e normalização
        out1 = self.layernorm1(inputs + attn_output)

        # 2. Feed-Forward + Conexão Residual
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # Conexão residual e normalização
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
        })
        return config

### 6.3 - Camada do Decoder Transformer
class TransformerDecoderLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim

        # 1. Atenção Multi-Cabeça MASCARADA (Self-Attention)
        # O decoder só pode "ver" os tokens anteriores.
        self.att1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

        # 2. Atenção Multi-Cabeça (Cross-Attention)
        # O decoder "vê" a saída do encoder.
        self.att2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

        # Rede Feed-Forward
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        self.dropout3 = layers.Dropout(dropout_rate)

    def call(self, inputs, encoder_outputs, training=False):
        # `inputs` é o output do decoder até agora
        # `encoder_outputs` é o output final do encoder

        # 1. Self-Attention MASCARADA
        # use_causal_mask=True garante que a posição 'i' só atenda
        # às posições < 'i'.
        attn1 = self.att1(inputs, inputs, use_causal_mask=True)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(inputs + attn1)

        # 2. Cross-Attention (Encoder-Decoder Attention)
        # Query = out1 (do decoder)
        # Key/Value = encoder_outputs (do encoder)
        attn2 = self.att2(query=out1, value=encoder_outputs, key=encoder_outputs)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        # 3. Feed-Forward
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
        })
        return config

In [34]:
## -----------------------------------------------------------------
## Célula 7: Montagem do Modelo Transformer Completo
## -----------------------------------------------------------------

def build_transformer_model():
    # --- ENCODER ---
    # Input do Encoder (sentenças em Português)
    encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

    # Embedding de Posição do Encoder
    # Usamos MAX_SEQ_LENGTH aqui
    x = TokenAndPositionEmbedding(MAX_SEQ_LENGTH, VOCAB_SIZE, EMBED_DIM)(encoder_inputs)

    # Pilha de Camadas do Encoder
    # O output do encoder será a memória de "contexto"
    for _ in range(NUM_ENCODER_LAYERS):
        x = TransformerEncoderLayer(EMBED_DIM, NUM_HEADS, FF_DIM)(x)
    encoder_outputs = x # Saída final do Encoder

    # --- DECODER ---
    # Input do Decoder (sentenças em Inglês, deslocadas)
    decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")

    # Embedding de Posição do Decoder
    # Usamos MAX_SEQ_LENGTH + 1 aqui (por causa do [START]/[END])
    x = TokenAndPositionEmbedding(MAX_SEQ_LENGTH + 1, VOCAB_SIZE, EMBED_DIM)(decoder_inputs)

    # Pilha de Camadas do Decoder
    # O decoder recebe seu próprio input (x) e a saída do encoder
    for _ in range(NUM_DECODER_LAYERS):
        x = TransformerDecoderLayer(EMBED_DIM, NUM_HEADS, FF_DIM)(x, encoder_outputs)

    # --- SAÍDA FINAL ---
    # Camada Dense final para prever a próxima palavra no vocabulário
    # A saída terá shape (batch_size, seq_len, vocab_size)
    outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)

    # Cria o modelo Keras
    model = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)
    return model

# Instancia o modelo
transformer = build_transformer_model()

transformer.summary()

# Plotar o modelo (útil no Colab)
# keras.utils.plot_model(transformer, show_shapes=True, dpi=64)

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, None, 256) │  3,847,680 │ encoder_inputs[0… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 256) │  1,315,840 │ token_and_positi… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, None, 256) │  3,847,936 │ decoder_inputs[0… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 256) │  1,315,840 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, None, 256) │  2,368,256 │ token_and_positi… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, None, 256) │  2,368,256 │ transformer_deco… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, None,      │  3,855,000 │ transformer_deco… │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,918,808 (72.17 MB)

 Trainable params: 18,918,808 (72.17 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
## -----------------------------------------------------------------
## Célula 8: Treinamento do Modelo
## -----------------------------------------------------------------

print("\nIniciando o treinamento...")

# Compilamos o modelo
# Usamos "sparse_categorical_crossentropy" porque nossos alvos (Y)
# são inteiros (IDs de tokens), e não one-hot encoded.
transformer.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"], # A acurácia aqui é "por token"
)

# Treina o modelo
# Usamos o EarlyStopping para parar se a validação não melhorar
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
]

history = transformer.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks
)

print("\nTreinamento concluído.")


Iniciando o treinamento...
Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 160s 95ms/step - accuracy: 0.7920 - loss: 2.6965 - val_accuracy: 0.8745 - val_loss: 0.7879
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 115s 82ms/step - accuracy: 0.8854 - loss: 0.7137 - val_accuracy: 0.9115 - val_loss: 0.5313
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 114s 81ms/step - accuracy: 0.9188 - loss: 0.4840 - val_accuracy: 0.9308 - val_loss: 0.4100
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 114s 81ms/step - accuracy: 0.9362 - loss: 0.3626 - val_accuracy: 0.9390 - val_loss: 0.3497
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 114s 81ms/step - accuracy: 0.9462 - loss: 0.2887 - val_accuracy: 0.9438 - val_loss: 0.3144
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 115s 81ms/step - accuracy: 0.9534 - loss: 0.2378 - val_accuracy: 0.9467 - val_loss: 0.2932
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 115s 82ms/step - accuracy: 0.9587 - loss: 0.2005 - val_accuracy: 0.9489 - val_loss: 0.2810
Epoch 8/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━

In [36]:
## -----------------------------------------------------------------
## Célula 9: Inferência (Tradução)
## -----------------------------------------------------------------

# Para traduzir, precisamos de um loop auto-regressivo:
# 1. Enviar a sentença PT para o Encoder -> obter `encoder_outputs`
# 2. Iniciar o Decoder com o token [START]
# 3. Loop:
#    a. Prever o próximo token
#    b. Pegar o token com maior prob. (argmax)
#    c. Se for [END], parar.
#    d. Senão, adicionar o token à sequência do decoder e repetir.

# Mapeamentos de ID -> Palavra para o vocabulário de Inglês (Alvo)
target_vocab = target_vectorization.get_vocabulary()
index_to_word = dict(zip(range(len(target_vocab)), target_vocab))
word_to_index = dict(zip(target_vocab, range(len(target_vocab))))

# Índices dos tokens especiais
START_TOKEN_INDEX = word_to_index["[START]"]
END_TOKEN_INDEX = word_to_index["[END]"]

def translate(input_sentence):
    """Traduza uma sentença em Português para Inglês."""

    print(f"Input: {input_sentence}")

    # 1. Vetorizar a sentença de entrada (PT)
    tokenized_input = source_vectorization([input_sentence])

    # 2. Iniciar a sequência do decoder. Começa apenas com [START].
    # Shape é (batch_size, seq_len) -> (1, 1)
    decoder_input_tokens = tf.convert_to_tensor([[START_TOKEN_INDEX]], dtype=tf.int64)

    # 3. Loop auto-regressivo
    for i in range(MAX_SEQ_LENGTH):
        # Obter as predições do modelo
        # O modelo é chamado com [encoder_inputs, decoder_inputs]
        preds = transformer([tokenized_input, decoder_input_tokens])

        # Pegar os logits apenas do ÚLTIMO token previsto
        # preds shape: (1, seq_len, vocab_size) -> (1, vocab_size)
        last_token_pred = preds[:, -1, :]

        # Encontrar o ID do token com a maior probabilidade (argmax)
        sampled_token_index = tf.argmax(last_token_pred, axis=1)[0].numpy()

        # Se for o token [END], paramos a tradução
        if sampled_token_index == END_TOKEN_INDEX:
            break

        # Adicionar o novo token previsto à sequência de entrada do decoder
        # para a próxima iteração
        decoder_input_tokens = tf.concat(
            [decoder_input_tokens, [[sampled_token_index]]], axis=1
        )

    # 4. Decodificar a sequência de tokens de saída
    output_tokens = decoder_input_tokens[0].numpy()

    # Converte os IDs de volta para palavras, ignorando [START]
    translated_text = " ".join(
        [index_to_word[token] for token in output_tokens
         if token not in [START_TOKEN_INDEX, END_TOKEN_INDEX]]
    )

    print(f"Output: {translated_text}\n")
    return translated_text

In [37]:
## -----------------------------------------------------------------
## Célula 10: Teste do Modelo
## -----------------------------------------------------------------

print("\n--- Testando Traduções (Exemplos de Validação) ---")

for i in range(10):
    pair = val_pairs[i]
    pt_sentence = pair[0]
    en_sentence = pair[1]

    translate(pt_sentence)
    print(f"(Referência: {en_sentence})\n" + ("-"*30))

print("\n--- Teste com Novas Sentenças ---")
translate("Eu gosto de gatos.")
translate("Este é um teste.")
translate("Onde fica o banheiro?")
translate("Quantos anos você tem?")


--- Testando Traduções (Exemplos de Validação) ---
Input: Os cavalos se assustam facilmente.
Output: horses scare yourself

(Referência: Horses are easily spooked.)
------------------------------
Input: É quase impossível aprender uma língua estrangeira em pouco tempo.
Output: its almost impossible to learn a foreign language in a while

(Referência: It is almost impossible to learn a foreign language in a short time.)
------------------------------
Input: É assim que nós fazemos as coisas.
Output: thats how we do things

(Referência: That's how we do things.)
------------------------------
Input: Quanto tempo você vai ficar em Boston?
Output: how long are you going to stay in boston

(Referência: How much time will you be in Boston?)
------------------------------
Input: Tente ter uma mente aberta.
Output: try to have a open open

(Referência: Try to have an open mind.)
------------------------------
Input: Diga tchau aos seus amigos.
Output: tell your friends to your friends

(Refer

'how old are you'